In [1]:
import findspark
findspark.init()

In [2]:
spark_url = 'local'

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, StructType

In [4]:
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Clean Scraped Data')\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/03 22:21:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/05/03 22:21:46 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [17]:
sc = spark.sparkContext

In [5]:
api_key = "11c74ac47ee301629d4f6f3a51e99a60"


In [10]:
import requests
import json

In [115]:
url = f"https://api.elsevier.com/content/search/scopus?query=all(Chulalongkorn)&date=2017&view=COMPLETE&apiKey={api_key}&count=5"
headers = {'content-type': 'applicaton/json'}

res = requests.get(url, headers=headers)

entries = res.json()["search-results"]["entry"]

scopus_ids = [ entry["dc:identifier"] for entry in entries]
print(scopus_ids)

['SCOPUS_ID:85029781991', 'SCOPUS_ID:85049840335', 'SCOPUS_ID:85039703338', 'SCOPUS_ID:85039924885', 'SCOPUS_ID:85039924338']


In [129]:
# for id in scopus_ids:
url = f"https://api.elsevier.com/content/abstract/scopus_id/SCOPUS_ID:85029781991?view=FULL&httpAccess=application%2Fjson&apiKey={api_key}"
res = requests.get(url)
print(res.text)

df = spark.read.option("multiline", True).json(sc.parallelize([res.text]))

df.printSchema()



<abstracts-retrieval-response xmlns="http://www.elsevier.com/xml/svapi/abstract/dtd" xmlns:dn="http://www.elsevier.com/xml/svapi/abstract/dtd" xmlns:ait="http://www.elsevier.com/xml/ani/ait" xmlns:ce="http://www.elsevier.com/xml/ani/common" xmlns:cto="http://www.elsevier.com/xml/cto/dtd" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:prism="http://prismstandard.org/namespaces/basic/2.0/" xmlns:xocs="http://www.elsevier.com/xml/xocs/dtd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><coredata><prism:url>https://api.elsevier.com/content/abstract/scopus_id/85029781991</prism:url><dc:identifier>SCOPUS_ID:85029781991</dc:identifier><eid>2-s2.0-85029781991</eid><pii>S0950061817318585</pii><prism:doi>10.1016/j.conbuildmat.2017.09.057</prism:doi><dc:title>Shear capacity of high performance fiber reinforced concrete I-beams</dc:title><prism:aggregationType>Journal</prism:aggregationType><srctype>j</srctype><subtype>ar</subtype><subtypeDescription>Article</subtypeDescription><citedby-

In [91]:
df = spark.read.json(sc.parallelize([res.text]))

df.printSchema()

root
 |-- search-results: struct (nullable = true)
 |    |-- entry: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- @_fa: string (nullable = true)
 |    |    |    |-- affiliation: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- @_fa: string (nullable = true)
 |    |    |    |    |    |-- affiliation-city: string (nullable = true)
 |    |    |    |    |    |-- affiliation-country: string (nullable = true)
 |    |    |    |    |    |-- affiliation-url: string (nullable = true)
 |    |    |    |    |    |-- affilname: string (nullable = true)
 |    |    |    |    |    |-- afid: string (nullable = true)
 |    |    |    |-- article-number: string (nullable = true)
 |    |    |    |-- authkeywords: string (nullable = true)
 |    |    |    |-- author: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- @_fa: stri

In [92]:
df = df.withColumn("entry", F.explode(df["search-results.entry"]))
df = df.drop("search-results")

df.show()
df.printSchema()

+--------------------+
|               entry|
+--------------------+
|{true, [{true, Is...|
|{true, [{true, Ba...|
|{true, [{true, Ba...|
|{true, [{true, Ba...|
|{true, [{true, Ba...|
+--------------------+

root
 |-- entry: struct (nullable = true)
 |    |-- @_fa: string (nullable = true)
 |    |-- affiliation: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- @_fa: string (nullable = true)
 |    |    |    |-- affiliation-city: string (nullable = true)
 |    |    |    |-- affiliation-country: string (nullable = true)
 |    |    |    |-- affiliation-url: string (nullable = true)
 |    |    |    |-- affilname: string (nullable = true)
 |    |    |    |-- afid: string (nullable = true)
 |    |-- article-number: string (nullable = true)
 |    |-- authkeywords: string (nullable = true)
 |    |-- author: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- @_fa: string (nullable = true)
 |    |    |  

In [96]:
base_path = "entry"

df = (df.withColumn("title", F.col(base_path + ".dc:title"))
.withColumn("id", F.col(base_path + ".dc:identifier"))
.withColumn("citation_count", F.col(base_path + ".citedby-count"))
.withColumn("authors", F.col(base_path + ".author.authname"))
.withColumn("affiliatons", F.col(base_path + ".affiliation.affilname"))
)

df.show()
df.printSchema()

+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+
|               entry|               title|                  id|citation_count|             authors|         affiliatons|
+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+
|{true, [{true, Is...|Anti-Plasmodium v...|SCOPUS_ID:8519139...|             0|[Kuamsab N., Puta...|[Tokai University...|
|{true, [{true, Ba...|Comprehensive gen...|SCOPUS_ID:8519126...|             0|[Buthasane W., Sh...|[Siriraj Hospital...|
|{true, [{true, Ba...|Influence of phar...|SCOPUS_ID:8519105...|             0|[Sornsamdang G., ...|[Ramathibodi Hosp...|
|{true, [{true, Ba...|Anti-inflammatory...|SCOPUS_ID:8519103...|             0|[Kasetsuwan N., R...|[Ramkhamhaeng Uni...|
|{true, [{true, Ba...|Diverse effects o...|SCOPUS_ID:8519082...|             0|[Pichetkun V., Kh...|[Chulalongkorn Un...|
+--------------------+--

In [102]:
scopus_ids = df.select(F.col("id")).collect()

for id in scopus_ids:
    


[Row(id='SCOPUS_ID:85191399900'), Row(id='SCOPUS_ID:85191260059'), Row(id='SCOPUS_ID:85191056042'), Row(id='SCOPUS_ID:85191032017'), Row(id='SCOPUS_ID:85190822580')]
